# DDR Midterm: Sridhar R Ramanujam

In [52]:
import pandas as pd
import numpy as np
import requests
import mysql.connector
import bs4
import time
import re
import os

## Part 1

In [51]:
# Creating the header and using the appropriate URL to access the API and retrieve data which is converted to JSON

header = {'User-agent': 'Mozilla/5.0'}
url = 'http://numbersapi.com/0..99'
response = requests.get(url,header)
data = response.json()

# Printing in the mentioned format

for i in range(0,100):
    num = str(i)
    print(num.zfill(3),"-", data[num])

000 - 0 is the atomic number of the theoretical element tetraneutron.
001 - 1 is the number of dimensions of a line.
002 - 2 is the number of polynucleotide strands in a DNA double helix.
003 - 3 is the number of semicircular canals a human ear has.
004 - 4 is the number of chambers the mammalian heart consists of.
005 - 5 is the number of dots in a quincunx.
006 - 6 is the number of sides on a cube.
007 - 7 is the number of main stars in the constellations of the Big Dipper and Orion.
008 - 8 is the number of bits in a byte.
009 - 9 is the number of innings in a regulation, non-tied game of baseball.
010 - 10 is the highest score possible in Olympics gymnastics competitions.
011 - 11 is the number of pounds one gallon of pure maple syrup weighs.
012 - 12 is the number of months in a year.
013 - 13 is the number of colonies that formed the United States.
014 - 14 is the number of points in a proposed republican constitution of the United Kingdom.
015 - 15 is the approximate speed in mi

## Part 2

### 2 a

In the URL for the search, the variable name "LH_BIN" stands for whether or not our search results include Buy It Now items. We can make  LH_BIN = 1, in order to only display Buy It Now items in the search results. 

Similarly, there is the variable called "_ ipg" that stores the number of results to be displayed in a page. We can set _ ipg = 100 to have 100 search results in one page.

### 2 b

In [49]:
# Getting the list of sponsored and non-sponsored URLs and storing them in two text files

url_2='https://www.ebay.com/sch/i.html?_nkw=iphone+11+pro&rt=nc&LH_BIN=1&_ipg=100&_pgn='
sponsored = open("Sponsored.txt", "w")
non_sponsored = open("Non-Sponsored.txt", "w")
for i in range(1,11):
    time.sleep(3)
    num = str(i)
    new_url = url_2+num
    response = requests.get(new_url,header)
    soup = bs4.BeautifulSoup(response.text)
    items = soup.select("#srp-river-results > ul > li.s-item")
    for item in items:
        a = item.find("a")
        if not a:
            continue
        text = item.find("span").get_text()
        if re.search('.*S.*P.*O.*N.*S.*O.*R.*E.*D.*',text):  
            url_simplify = re.sub("(.*)\\?.*", r"\1", a['href']) 
            sponsored.write(url_simplify+"\n")
        else:
            url_simplify = re.sub("(.*)\\?.*", r"\1", a['href']) 
            non_sponsored.write(url_simplify+"\n")
sponsored.close()
non_sponsored.close()
print("done")

done


### 2 c

In [47]:
# Creating new folders for Sponsored and Non-Sponsored items using current working directory

directory = os.getcwd()
path1 = r'{}/Sponsored'.format(directory)
path2 = r'{}/Non-Sponsored'.format(directory) 
os.makedirs(path1)
os.makedirs(path2)

In [60]:
# Opening each URL from Sponsored.txt and saving the page in the Sponsored folder as <item_id>.htm files

filepath = 'Sponsored.txt'
with open(filepath) as fp:
    line = fp.readline()
    count = 1
    while line:
        time.sleep(2)
        url = line.strip()
        itm_id = re.sub("(.*)//?([0-9]+)", r"\2",url)
        response = requests.get(url,header)
        with open(path1+"/"+itm_id+".htm", 'w') as file:
            file.write(response.text)
            file.close()
        if count%10==0:
            print(count) # Printing to track progress
        line = fp.readline()
        count += 1

10
20
30
40
50
60


In [61]:
# Opening each URL from Non-Sponsored.txt and saving the page in the Non-Sponsored folder as <item_id>.htm files

filepath = 'Non-Sponsored.txt'
with open(filepath) as fp:
    line = fp.readline()
    count = 1
    while line:
        time.sleep(2)
        url = line.strip()
        itm_id = re.sub("(.*)//?([0-9]+)", r"\2",url)
        response = requests.get(url,header)
        with open(path2+"/"+itm_id+".htm", 'w') as file:
            file.write(response.text)
            file.close()
        if count%50==0:
            print(count) # Printing to track progress
        line = fp.readline()
        count += 1

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000


### 2 d

In [6]:
# Creating a list of all the filenames from the Sponsored folder

from os import walk

pages_1 = []
for (dirpath, dirnames, filenames) in walk(path1):
    pages_1.extend(filenames)
    break
pages_1.remove(".DS_Store") # removing random unexpected filename

In [7]:
# Creating a list of all the filenames from the Non-Sponsored folder

pages_2 = []
for (dirpath, dirnames, filenames) in walk(path2):
    pages_2.extend(filenames)
    break

In [8]:
# creating empty arrays for the different data points we need to extract

sponsorship_1 = []
seller_name_1 = []
seller_score_1 = []
item_currency_1 = []
item_price_1 = []
items_sold_1 = []
best_offer_1 = []
title_1 = []
returns_allowed_1 = []
shipping_calc_1 = []
shipping_currency_1 = []
shipping_1 = []
condition_1 = []

# running a loop through all the sponsored pages 

for item in range(0,len(pages_1)):
    filename = pages_1[item]
    with open(path1+"/"+filename, 'r') as file:
        text = file.read()
        soup = bs4.BeautifulSoup(text, 'html.parser') 
# Selecting the seller name
        seller_name_1.append(soup.find("span", class_ = "mbg-nw").get_text())
# Selecting the seller score
        seller_score_1.append(int(re.sub(".*feedback score: ([0-9]+).*",r"\1",str(soup.select("span.mbg-l > a")))))
# Selecting the item price and currency
        p = re.sub(".??([0-9]*)(,*)([0-9]+)\.([0-9]+)",r"\1\3\4",str(soup.find("span",class_ = "notranslate").get_text()))
        c = re.sub("([A-Z]+).([0-9]+)",r"\1",p.strip("\n").strip("\t").strip("/ea"))
        if(re.sub("(.).*",r"\1",c).isalpha() == False):
            item_currency_1.append("US")
        else:
            item_currency_1.append(c)
        p1 = re.sub("[A-Z]+ ([0-9]+)",r"\1",p.strip("\n").strip("\t"))
        if(re.sub("(.).*",r"\1",p1).isalpha() == True):
            item_price_1.append(int(re.sub("...([0-9]+)",r"\1",p1)))
        else:
            item_price_1.append(int(re.sub("([0-9]+).*",r"\1",p1)))
# Selecting the number of items sold
        try:
            items_sold_1.append(int(re.sub("([0-9]+) .*",r"\1",soup.find("a",class_ = "vi-txt-underline").get_text())))
        except:
            items_sold_1.append(None)
# Checking whether there is a best offer available
        try:
            check = soup.find("div",class_ = "vi-bbox-dspn u-flL lable boLable").get_text()
        except:
            check = "NA"
        if (check == "NA"):
            best_offer_1.append(0)
        else:
            best_offer_1.append(1)
# Selecting the title of the item        
        title_1.append(soup.find("span",id = "vi-lkhdr-itmTitl").get_text())
# Checking whether there is a possibility for returns
        try:
            if (soup.find("span", id = "vi-ret-accrd-txt").get_text() == "Seller does not accept returns"):
                returns_allowed_1.append(0)
            else:
                returns_allowed_1.append(1)
        except:
            returns_allowed_1.append(None)
# Selecting shipping price and currency
        try:
            try:
                if(re.sub("<span>(.*)</span>",r"\1",str(soup.select("span#fshippingCost > span")[0]))=="FREE"):
                    shipping_currency_1.append("US")
                    shipping_1.append(0)
                    shipping_calc_1.append(0)
                else: 
                    s = re.sub("<span>(.*)</span>",r"\1",str(soup.select("span#fshippingCost > span")[0]))
                    s1 = re.sub("([A-Z]+).*",r"\1",s)
                    if (re.sub("(.)[0-9].*",r"\1",s1)=="$"):
                        shipping_currency_1.append("US")
                    else:
                        shipping_currency_1.append(s1)
                    s2 = re.sub("....([0-9]+)\.([0-9]+)",r"\1\2",s)
                    if (re.sub("(.)[0-9].*",r"\1",s2)=="$"):
                        shipping_1.append(int(re.sub(".([0-9]+)\.([0-9]+)",r"\1\2",s2)))
                    else:
                        shipping_1.append(int(s2))
                    shipping_calc_1.append(0)
            except:
                if(soup.find("strong", class_ = "sh_gr_bld").get_text()=="FAST 'N FREE"):
                    shipping_currency_1.append("US")
                    shipping_1.append(0)
                    shipping_calc_1.append(0)
        except:
            shipping_currency_1.append(None)
            shipping_1.append(None)
            shipping_calc_1.append(1)
# Selecting the condition of the item
        condition_1.append(soup.find("div", itemprop = "itemCondition").get_text())
# Assigning that the item is sponsored
        sponsorship_1.append("Sponsored")
print("done")

done


In [18]:
# creating empty arrays for the different data points we need to extract

sponsorship_2 = []
seller_name_2 = []
seller_score_2 = []
item_currency_2 = []
item_price_2 = []
items_sold_2 = []
best_offer_2 = []
title_2 = []
returns_allowed_2 = []
shipping_currency_2 = []
shipping_calc_2 = []
shipping_2 = []
condition_2 = []

# running a loop through all the non-sponsored pages

for item in range(0,len(pages_2)):
    filename = pages_2[item]
    with open(path2+"/"+filename, 'r') as file:
        text = file.read()
        soup = bs4.BeautifulSoup(text, 'html.parser') 
# Selecting the seller name
        seller_name_2.append(soup.find("span", class_ = "mbg-nw").get_text())
# Selecting the seller score
        seller_score_2.append(int(re.sub(".*feedback score: ([0-9]+).*",r"\1",str(soup.select("span.mbg-l > a")))))
# Selecting the item price and currency
        p = re.sub(".??([0-9]*)(,*)([0-9]+)\.([0-9]+)",r"\1\3\4",str(soup.find("span",class_ = "notranslate").get_text()))
        c = re.sub("([A-Z]+).([0-9]+)",r"\1",p.strip("\n").strip("\t").strip("/ea"))
        if(re.sub("(.).*",r"\1",c).isalpha() == False):
            item_currency_2.append("US")
        else:
            item_currency_2.append(c)
        p1 = re.sub("[A-Z]+ ([0-9]+)",r"\1",p.strip("\n").strip("\t"))
        if(re.sub("(.).*",r"\1",p1).isalpha() == True):
            item_price_2.append(int(re.sub("...([0-9]+)",r"\1",p1)))
        else:
            item_price_2.append(int(re.sub("([0-9]+).*",r"\1",p1)))
# Selecting the number of items sold
        try:
            items_sold_2.append(int(re.sub("([0-9]+) .*",r"\1",soup.find("a",class_ = "vi-txt-underline").get_text())))
        except:
            items_sold_2.append(None)
# Checking whether there is a best offer available
        try:
            check = soup.find("div",class_ = "vi-bbox-dspn u-flL lable boLable").get_text()
        except:
            check = "NA"
        if (check == "NA"):
            best_offer_2.append(0)
        else:
            best_offer_2.append(1)
# Selecting the title of the item
        title_2.append(soup.find("span",id = "vi-lkhdr-itmTitl").get_text())
# Checking whether there is a possibility for returns
        try:
            if (soup.find("span", id = "vi-ret-accrd-txt").get_text() == "Seller does not accept returns"):
                returns_allowed_2.append(0)
            else:
                returns_allowed_2.append(1)
        except:
            returns_allowed_2.append(None)
# Selecting shipping price and currency
        try:
            try:
                if(re.sub("<span>(.*)</span>",r"\1",str(soup.select("span#fshippingCost > span")[0]))=="FREE"):
                    shipping_currency_2.append("US")
                    shipping_2.append(0)
                    shipping_calc_2.append(0)
                else: 
                    s = re.sub("<span>(.*)</span>",r"\1",str(soup.select("span#fshippingCost > span")[0]))
                    s1 = re.sub("([A-Z]+).*",r"\1",s)
                    if (re.sub("(.)[0-9].*",r"\1",s1)=="$"):
                        shipping_currency_2.append("US")
                    else:
                        shipping_currency_2.append(s1)
                    s2 = re.sub("....([0-9]+)\.([0-9]+)",r"\1\2",s)
                    if (re.sub("(.)[0-9].*",r"\1",s2)=="$"):
                        shipping_2.append(int(re.sub(".([0-9]+)\.([0-9]+)",r"\1\2",s2)))
                    else:
                        shipping_2.append(int(s2))
                    shipping_calc_2.append(0)
            except:
                if(soup.find("strong", class_ = "sh_gr_bld").get_text()=="FAST 'N FREE"):
                    shipping_currency_2.append("US")
                    shipping_2.append(0)
                    shipping_calc_2.append(0)
        except:
            shipping_currency_2.append(None)
            shipping_2.append(None)
            shipping_calc_2.append(1)

# Selecting the condition of the item
        condition_2.append(soup.find("div", itemprop = "itemCondition").get_text())
# Assigning that the item is non-sponsored
        sponsorship_2.append("Non-Sponsored")
print("done")

done


### 2 e

In [9]:
# Connecting to mysql server

mydb = mysql.connector.connect(user='root',password = '',
                              host='localhost')

In [10]:
# Creating a databse called 'eBay'

mycursor = mydb.cursor()
mycursor.execute("Create database if not exists eBay")
mycursor.execute("SHOW DATABASES")
for x in mycursor:
    print(x)

('eBay',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('ucdavis',)


In [11]:
# Connecting to the 'eBay' database

mydb = mysql.connector.connect(user='root',password = '',
                              host='localhost', database = 'eBay')

In [12]:
# Creating a new table called 'eBay_items with the required column names'

mycursor = mydb.cursor()
mycursor.execute("drop table eBay_items")
mycursor.execute("Create table if not exists eBay_items (Sponsorship VARCHAR (25), Title VARCHAR (200), Price_Currency VARCHAR (10), Price_Cents INT, Seller_Name VARCHAR (100), Seller_Score INT, Num_Items_Sold INT, Best_Offer_Available INT, Returns_Allowed INT, Shipping_Calc INT, Shipping_Currency VARCHAR (10), Shipping_Price_Cents INT, Item_Condition VARCHAR (100))")


In [13]:
# Checking if table has been created

mycursor.execute("show tables;")
for x in mycursor:
    print(x)

('eBay_items',)


In [14]:
# Populating the table with data for sponsored items

for i in range(0,len(pages_1)):
    sql = "INSERT INTO eBay_items VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val = (sponsorship_1[i], title_1[i], item_currency_1[i], item_price_1[i], seller_name_1[i], seller_score_1[i], items_sold_1[i], best_offer_1[i], returns_allowed_1[i], shipping_calc_1[i], shipping_currency_1[i], shipping_1[i], condition_1[i])
    mycursor.execute(sql,val)
    mydb.commit()
    

In [19]:
# Populating the table with data for non-sponsored items

for i in range(0,len(pages_2)):
    sql = "INSERT INTO eBay_items VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val = (sponsorship_2[i], title_2[i], item_currency_2[i], item_price_2[i], seller_name_2[i], seller_score_2[i], items_sold_2[i], best_offer_2[i], returns_allowed_2[i], shipping_calc_2[i], shipping_currency_2[i], shipping_2[i], condition_2[i])
    mycursor.execute(sql,val)
    mydb.commit()
    

### 2 f

In [28]:
# Getting mean, min, max and standard deviation stats for item price grouped by sponsorship status and condition

mycursor = mydb.cursor()

mycursor.execute("SELECT Sponsorship, Item_Condition, avg(Price_Cents), min(Price_Cents), max(Price_Cents), stddev(Price_Cents) FROM eBay_items group by Sponsorship, Item_Condition")
y = mycursor.fetchall()
for x in y:
    print(x)

('Sponsored', 'Seller refurbished', Decimal('117500.0000'), 117500, 117500, 0.0)
('Sponsored', 'New', Decimal('171364.6744'), 510, 510000, 129459.8151631619)
('Sponsored', 'Manufacturer refurbished', Decimal('104000.0000'), 104000, 104000, 0.0)
('Sponsored', 'Open box', Decimal('147799.7143'), 84999, 234900, 53119.67109599738)
('Sponsored', 'For parts or not working', Decimal('83484.5714'), 56999, 115000, 17599.52256541672)
('Sponsored', 'Used', Decimal('99949.5000'), 89999, 109900, 9950.5)
('Non-Sponsored', 'Used', Decimal('101635.0938'), 50000, 229900, 26793.133022240254)
('Non-Sponsored', 'New', Decimal('86418.5660'), 550, 799999, 113735.06793439873)
('Non-Sponsored', 'For parts or not working', Decimal('75154.9386'), 36800, 119900, 16163.910528827757)
('Non-Sponsored', 'Seller refurbished', Decimal('115241.0909'), 82999, 144900, 14556.676570030082)
('Non-Sponsored', 'Open box', Decimal('116149.0265'), 600, 218296, 36905.4941993281)
('Non-Sponsored', 'Manufacturer refurbished', Deci

In [29]:
# Getting mean, min, max and standard deviation stats for seller score grouped by sponsorship status and condition

mycursor = mydb.cursor()

mycursor.execute("SELECT Sponsorship, Item_Condition, avg(Seller_Score), min(Seller_Score), max(Seller_Score), stddev(Seller_Score) FROM eBay_items group by Sponsorship, Item_Condition")
y = mycursor.fetchall()
for x in y:
    print(x)

('Sponsored', 'Seller refurbished', Decimal('275.5000'), 253, 298, 22.5)
('Sponsored', 'New', Decimal('8013.6279'), 4, 60473, 12438.918916651684)
('Sponsored', 'Manufacturer refurbished', Decimal('298.0000'), 298, 298, 0.0)
('Sponsored', 'Open box', Decimal('2556.4286'), 1, 8650, 3854.582499428175)
('Sponsored', 'For parts or not working', Decimal('11784.4286'), 127, 75626, 26084.146317294744)
('Sponsored', 'Used', Decimal('4783.5000'), 3587, 5980, 1196.5)
('Non-Sponsored', 'Used', Decimal('27604.9531'), 0, 258707, 61973.272028761945)
('Non-Sponsored', 'New', Decimal('3404.2944'), 0, 258704, 12273.178495678369)
('Non-Sponsored', 'For parts or not working', Decimal('4589.3509'), 3, 46063, 9008.595748956286)
('Non-Sponsored', 'Seller refurbished', Decimal('279296.3864'), 68, 2264744, 446708.414907065)
('Non-Sponsored', 'Open box', Decimal('14654.4425'), 0, 325971, 43956.30443832841)
('Non-Sponsored', 'Manufacturer refurbished', Decimal('836419.6667'), 1511, 2264744, 1014778.3068750644)


In [30]:
# Getting mean, min, max and standard deviation stats for # items sold grouped by sponsorship status and condition

mycursor = mydb.cursor()

mycursor.execute("SELECT Sponsorship, Item_Condition, avg(Num_Items_Sold), min(Num_Items_Sold), max(Num_Items_Sold), stddev(Num_Items_Sold) FROM eBay_items group by Sponsorship, Item_Condition")
y = mycursor.fetchall()
for x in y:
    print(x)

('Sponsored', 'Seller refurbished', Decimal('1.0000'), 1, 1, 0.0)
('Sponsored', 'New', Decimal('6.7143'), 1, 61, 12.93258290830846)
('Sponsored', 'Manufacturer refurbished', None, None, None, None)
('Sponsored', 'Open box', Decimal('85.0000'), 1, 303, 126.62543188475212)
('Sponsored', 'For parts or not working', Decimal('1.0000'), 1, 1, 0.0)
('Sponsored', 'Used', None, None, None, None)
('Non-Sponsored', 'Used', Decimal('1.3333'), 1, 3, 0.6666666666666666)
('Non-Sponsored', 'New', Decimal('7.6798'), 1, 359, 22.5431479116462)
('Non-Sponsored', 'For parts or not working', Decimal('3.1667'), 1, 11, 3.531603350069515)
('Non-Sponsored', 'Seller refurbished', Decimal('6.7333'), 1, 29, 9.574038971208662)
('Non-Sponsored', 'Open box', Decimal('17.2308'), 1, 150, 39.35275153346253)
('Non-Sponsored', 'Manufacturer refurbished', Decimal('108.0000'), 108, 108, 0.0)


In [31]:
# Getting mean, min, max and standard deviation stats for shipping price grouped by sponsorship status and condition

mycursor = mydb.cursor()

mycursor.execute("SELECT Sponsorship, Item_Condition, avg(Shipping_Price_Cents), min(Shipping_Price_Cents), max(Shipping_Price_Cents), stddev(Shipping_Price_Cents) FROM eBay_items group by Sponsorship, Item_Condition")
y = mycursor.fetchall()
for x in y:
    print(x)

('Sponsored', 'Seller refurbished', Decimal('0.0000'), 0, 0, 0.0)
('Sponsored', 'New', Decimal('5504.8293'), 0, 29999, 9529.3509053958)
('Sponsored', 'Manufacturer refurbished', Decimal('0.0000'), 0, 0, 0.0)
('Sponsored', 'Open box', Decimal('0.0000'), 0, 0, 0.0)
('Sponsored', 'For parts or not working', Decimal('100.0000'), 0, 700, 244.94897427831782)
('Sponsored', 'Used', Decimal('649.5000'), 0, 1299, 649.5)
('Non-Sponsored', 'Used', Decimal('632.5965'), 0, 19900, 2646.8133362934136)
('Non-Sponsored', 'New', Decimal('482.7735'), 0, 29999, 1735.3684108213827)
('Non-Sponsored', 'For parts or not working', Decimal('235.4752'), 0, 2499, 527.3251123735623)
('Non-Sponsored', 'Seller refurbished', Decimal('63.2273'), 0, 949, 233.87835451521332)
('Non-Sponsored', 'Open box', Decimal('613.4804'), 0, 2980, 1027.1686072182833)
('Non-Sponsored', 'Manufacturer refurbished', Decimal('525.0000'), 0, 1575, 742.4621202458749)


In [32]:
# Getting mean, min, max and standard deviation stats for best offer availability grouped by sponsorship status and condition

mycursor = mydb.cursor()

mycursor.execute("SELECT Sponsorship, Item_Condition, avg(Best_Offer_Available), min(Best_Offer_Available), max(Best_Offer_Available), stddev(Best_Offer_Available) FROM eBay_items group by Sponsorship, Item_Condition")
y = mycursor.fetchall()
for x in y:
    print(x)

('Sponsored', 'Seller refurbished', Decimal('0.0000'), 0, 0, 0.0)
('Sponsored', 'New', Decimal('0.2093'), 0, 1, 0.40681059731478836)
('Sponsored', 'Manufacturer refurbished', Decimal('0.0000'), 0, 0, 0.0)
('Sponsored', 'Open box', Decimal('0.8571'), 0, 1, 0.3499271061118826)
('Sponsored', 'For parts or not working', Decimal('0.1429'), 0, 1, 0.3499271061118826)
('Sponsored', 'Used', Decimal('0.5000'), 0, 1, 0.5)
('Non-Sponsored', 'Used', Decimal('0.2031'), 0, 1, 0.4023247871744916)
('Non-Sponsored', 'New', Decimal('0.1077'), 0, 1, 0.31005049054714695)
('Non-Sponsored', 'For parts or not working', Decimal('0.2807'), 0, 1, 0.4493420517496735)
('Non-Sponsored', 'Seller refurbished', Decimal('0.0000'), 0, 0, 0.0)
('Non-Sponsored', 'Open box', Decimal('0.2832'), 0, 1, 0.4505459136763842)
('Non-Sponsored', 'Manufacturer refurbished', Decimal('0.0000'), 0, 0, 0.0)


In [33]:
# Getting mean, min, max and standard deviation stats for return policy grouped by sponsorship status and condition

mycursor = mydb.cursor()

mycursor.execute("SELECT Sponsorship, Item_Condition, avg(Returns_Allowed), min(Returns_Allowed), max(Returns_Allowed), stddev(Returns_Allowed) FROM eBay_items group by Sponsorship, Item_Condition")
y = mycursor.fetchall()
for x in y:
    print(x)

('Sponsored', 'Seller refurbished', Decimal('0.5000'), 0, 1, 0.5)
('Sponsored', 'New', Decimal('0.3953'), 0, 1, 0.4889254893402008)
('Sponsored', 'Manufacturer refurbished', Decimal('1.0000'), 1, 1, 0.0)
('Sponsored', 'Open box', Decimal('0.2857'), 0, 1, 0.45175395145262565)
('Sponsored', 'For parts or not working', Decimal('0.2857'), 0, 1, 0.45175395145262565)
('Sponsored', 'Used', Decimal('1.0000'), 1, 1, 0.0)
('Non-Sponsored', 'Used', Decimal('0.5625'), 0, 1, 0.49607837082461076)
('Non-Sponsored', 'New', Decimal('0.7219'), 0, 1, 0.44807060753208405)
('Non-Sponsored', 'For parts or not working', Decimal('0.3596'), 0, 1, 0.47989752163473437)
('Non-Sponsored', 'Seller refurbished', Decimal('0.9545'), 0, 1, 0.20829889522526543)
('Non-Sponsored', 'Open box', Decimal('0.6637'), 0, 1, 0.4724370907978456)
('Non-Sponsored', 'Manufacturer refurbished', Decimal('0.6667'), 0, 1, 0.4714045207910317)


In [34]:
# Getting mean, min, max and standard deviation stats for variable shipping policy grouped by sponsorship status and condition

mycursor = mydb.cursor()

mycursor.execute("SELECT Sponsorship, Item_Condition, avg(Shipping_Calc), min(Shipping_Calc), max(Shipping_Calc), stddev(Shipping_Calc) FROM eBay_items group by Sponsorship, Item_Condition")
y = mycursor.fetchall()
for x in y:
    print(x)

('Sponsored', 'Seller refurbished', Decimal('0.5000'), 0, 1, 0.5)
('Sponsored', 'New', Decimal('0.0465'), 0, 1, 0.21059035204970736)
('Sponsored', 'Manufacturer refurbished', Decimal('0.0000'), 0, 0, 0.0)
('Sponsored', 'Open box', Decimal('0.2857'), 0, 1, 0.45175395145262565)
('Sponsored', 'For parts or not working', Decimal('0.0000'), 0, 0, 0.0)
('Sponsored', 'Used', Decimal('0.0000'), 0, 0, 0.0)
('Non-Sponsored', 'Used', Decimal('0.1094'), 0, 1, 0.3121091305537215)
('Non-Sponsored', 'New', Decimal('0.0486'), 0, 1, 0.2149430192367317)
('Non-Sponsored', 'For parts or not working', Decimal('0.1140'), 0, 1, 0.3178538759998849)
('Non-Sponsored', 'Seller refurbished', Decimal('0.0000'), 0, 0, 0.0)
('Non-Sponsored', 'Open box', Decimal('0.0973'), 0, 1, 0.2964271544149421)
('Non-Sponsored', 'Manufacturer refurbished', Decimal('0.0000'), 0, 0, 0.0)


In [35]:
# Getting counts of each level of item currency grouped by sponsorship status and condition

mycursor = mydb.cursor()

mycursor.execute("SELECT Sponsorship, Item_Condition, Price_Currency, count(Price_Currency) FROM eBay_items group by Sponsorship, Item_Condition, Price_Currency")
y = mycursor.fetchall()
for x in y:
    print(x)

('Sponsored', 'Seller refurbished', 'US', 2)
('Sponsored', 'New', 'US', 27)
('Sponsored', 'New', 'AU', 15)
('Sponsored', 'Manufacturer refurbished', 'US', 1)
('Sponsored', 'Open box', 'US', 7)
('Sponsored', 'For parts or not working', 'US', 7)
('Sponsored', 'Used', 'US', 2)
('Sponsored', 'New', 'GBP', 1)
('Non-Sponsored', 'Used', 'US', 63)
('Non-Sponsored', 'New', 'US', 633)
('Non-Sponsored', 'For parts or not working', 'US', 114)
('Non-Sponsored', 'Seller refurbished', 'US', 44)
('Non-Sponsored', 'Open box', 'US', 100)
('Non-Sponsored', 'New', 'AU', 21)
('Non-Sponsored', 'Open box', 'AU', 13)
('Non-Sponsored', 'New', 'C', 2)
('Non-Sponsored', 'New', 'GBP', 3)
('Non-Sponsored', 'Manufacturer refurbished', 'US', 3)
('Non-Sponsored', 'Used', 'AU', 1)


In [37]:
# Getting counts of each level of shipping currency grouped by sponsorship status and condition

mycursor = mydb.cursor()

mycursor.execute("SELECT Sponsorship, Item_Condition, Shipping_Currency, count(Shipping_Currency) FROM eBay_items group by Sponsorship, Item_Condition, Shipping_Currency")
y = mycursor.fetchall()
for x in y:
    print(x)

('Sponsored', 'Seller refurbished', None, 0)
('Sponsored', 'New', 'US', 25)
('Sponsored', 'New', 'AU', 15)
('Sponsored', 'Manufacturer refurbished', 'US', 1)
('Sponsored', 'Open box', None, 0)
('Sponsored', 'For parts or not working', 'US', 7)
('Sponsored', 'Open box', 'US', 5)
('Sponsored', 'New', None, 0)
('Sponsored', 'Used', 'US', 2)
('Sponsored', 'New', 'GBP', 1)
('Sponsored', 'Seller refurbished', 'US', 1)
('Non-Sponsored', 'Used', 'US', 56)
('Non-Sponsored', 'New', 'US', 601)
('Non-Sponsored', 'For parts or not working', 'US', 101)
('Non-Sponsored', 'New', None, 0)
('Non-Sponsored', 'Seller refurbished', 'US', 44)
('Non-Sponsored', 'For parts or not working', None, 0)
('Non-Sponsored', 'Open box', 'US', 89)
('Non-Sponsored', 'New', 'AU', 21)
('Non-Sponsored', 'Open box', 'AU', 13)
('Non-Sponsored', 'Used', None, 0)
('Non-Sponsored', 'Open box', None, 0)
('Non-Sponsored', 'New', 'C', 2)
('Non-Sponsored', 'New', 'GBP', 3)
('Non-Sponsored', 'Manufacturer refurbished', 'US', 3)
('No

In [40]:
# Getting counts of each level of seller name grouped by sponsorship status and condition

mycursor = mydb.cursor()

mycursor.execute("SELECT Sponsorship, Item_Condition, Seller_Name, count(Seller_Name) FROM eBay_items group by Sponsorship, Item_Condition, Seller_Name order by count(Seller_Name) desc")
y = mycursor.fetchall()
for x in y:
    print(x)

('Non-Sponsored', 'New', 'omrixus', 56)
('Non-Sponsored', 'New', 'aksmartdeals1', 54)
('Non-Sponsored', 'New', 'boazmoav', 35)
('Non-Sponsored', 'Seller refurbished', 'vipoutlet', 29)
('Non-Sponsored', 'New', 'adi_visler', 27)
('Non-Sponsored', 'New', 'purpletechnado', 22)
('Non-Sponsored', 'New', 'aurum-edition', 19)
('Non-Sponsored', 'New', 'omle_62', 17)
('Non-Sponsored', 'New', 'smartshophk', 16)
('Non-Sponsored', 'New', 'bq_shop', 15)
('Non-Sponsored', 'New', 'kim-downtown', 14)
('Non-Sponsored', 'New', 'trinityconnectonline', 14)
('Non-Sponsored', 'Open box', 'purpletechnado', 13)
('Non-Sponsored', 'Open box', 'allphones_online', 13)
('Non-Sponsored', 'New', 'tomesavi_4', 13)
('Non-Sponsored', 'Open box', 'itsworthmore', 13)
('Non-Sponsored', 'New', 'dealwithmenow', 12)
('Sponsored', 'New', 'narrowgate15', 10)
('Sponsored', 'New', 'be-start', 10)
('Non-Sponsored', 'New', 'yossefhadad18', 10)
('Non-Sponsored', 'New', 'super.fast.shop', 9)
('Non-Sponsored', 'New', 'kika676', 9)
('N

### 2 g

The prices of sponsored items tend to be slightly higher and more variable than non-sponsored items

The seller scores of non-sponsored items are much better on an average as compared to those of sponsored items

Sponsored items have a higher chance of having free shipping and generally has lower shipping costs

A much higher percentage of sponsored items are from outside the US in compasrison to non-sponsored items

Almost all the top users in terms of number of items listed sell non-sponsored items

#### If we have to predict whether an item is sponsored or not, the best columns to examine would be price, seller score, shipping costs and price currency to determine where the item is being sold from. 